Get device Dimension Data 

In [0]:
%pip install geopy Nominatim

In [0]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import udf

# Sample UK cities and reservoir names
uk_cities = ["London", "Manchester", "Birmingham", "Leeds", "Glasgow", "Southampton", "Liverpool", "Newcastle", "Nottingham", "Sheffield"]
reservoirs = ["Kielder Water", "Rutland Water", "Lake Vyrnwy", "Derwent Reservoir", "Pitsford Water", "Bewl Water", "Grafham Water", "Carsington Water", "Chew Valley Lake", "Ladybower Reservoir"]

# Generate dummy data
data = [Row(device_id=i+1, 
            location=uk_cities[i], 
            site=reservoirs[i], 
            site_manager_email=f"manager{i}@example.com") for i in range(10)]

# Create DataFrame
df = spark.createDataFrame(data)

pdf = df.toPandas()


# Display the DataFrame
display(df)



In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from geopy.geocoders import Nominatim


def get_long_lat(s):
  geolocator = Nominatim(user_agent="geo_lookup")
  country ="UK"
  loc = geolocator.geocode(s['location']+','+ country)
  s['longitude'] = loc.longitude
  s['latitude'] = loc.latitude
  return s


pdf1 = pdf.apply(get_long_lat, axis=1)

In [0]:
pdf1

In [0]:
df1 = spark.createDataFrame(pdf1)
df1.write.mode('overwrite').option('overwriteSchema', True).saveAsTable('alex_young.device_demo.device_dimensions')